# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
#teste
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [2]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [3]:
def get_files(filepath):
    all_files = []
    #os.walk da o caminho do diretório com os arquivos. https://www.geeksforgeeks.org/os-walk-python/
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [4]:
#get a list of all song JSON files in data/song_data
song_files = get_files('data/song_data')
print(song_files)

['/home/workspace/data/song_data/A/B/C/TRABCRU128F423F449.json', '/home/workspace/data/song_data/A/B/C/TRABCTK128F934B224.json', '/home/workspace/data/song_data/A/B/C/TRABCUQ128E0783E2B.json', '/home/workspace/data/song_data/A/B/C/TRABCIX128F4265903.json', '/home/workspace/data/song_data/A/B/C/TRABCYE128F934CE1D.json', '/home/workspace/data/song_data/A/B/C/TRABCEC128F426456E.json', '/home/workspace/data/song_data/A/B/C/TRABCXB128F4286BD3.json', '/home/workspace/data/song_data/A/B/C/TRABCAJ12903CDFCC2.json', '/home/workspace/data/song_data/A/B/C/TRABCPZ128F4275C32.json', '/home/workspace/data/song_data/A/B/C/TRABCKL128F423A778.json', '/home/workspace/data/song_data/A/B/C/TRABCEI128F424C983.json', '/home/workspace/data/song_data/A/B/C/TRABCFL128F149BB0D.json', '/home/workspace/data/song_data/A/B/B/TRABBBV128F42967D7.json', '/home/workspace/data/song_data/A/B/B/TRABBLU128F93349CF.json', '/home/workspace/data/song_data/A/B/B/TRABBXU128F92FEF48.json', '/home/workspace/data/song_data/A/B/B/T

In [5]:
#Select the first song in this list
filepath = song_files[0]
print(filepath)

/home/workspace/data/song_data/A/B/C/TRABCRU128F423F449.json


In [6]:
#Read the song file and view the data
df = pd.read_json(filepath, lines=True)
df.head()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,AR8IEZO1187B99055E,NaN,,NaN,Marc Shaiman,149.86404,1,SOINLJW12A8C13314C,City Slickers,2008


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [7]:

def get_song_data(song_files):
    df3 = pd.DataFrame()
    for i in song_files:
        df2 = pd.read_json(i, lines = True)
        df3 = df3.append(df2)
    df3 = df3[['song_id','title','artist_id','year','duration']]
    df3.values
    df3 = df3.drop_duplicates(keep = 'first')
    df3 = df3.values.tolist()
    return df3
song_data = get_song_data(song_files) 
print(song_data)

[['SOINLJW12A8C13314C', 'City Slickers', 'AR8IEZO1187B99055E', 2008, 149.86404], ['SOGDBUF12A8C140FAA', 'Intro', 'AR558FS1187FB45658', 2003, 75.67628], ['SORRZGD12A6310DBC3', 'Harajuku Girls', 'ARVBRGZ1187FB4675A', 2004, 290.55955], ['SONWXQJ12A8C134D94', 'The Ballad Of Sleeping Beauty', 'ARNF6401187FB57032', 1994, 305.162], ['SOFCHDR12AB01866EF', 'Living Hell', 'AREVWGE1187B9B890A', 0, 282.43546], ['SONSKXP12A8C13A2C9', 'Native Soul', 'AR0IAWL1187B9A96D0', 2003, 197.19791], ['SODAUVL12A8C13D184', 'Prognosis', 'ARWB3G61187FB49404', 2000, 363.85914], ['SOSWKAV12AB018FC91', 'Midnight Star', 'ARULZCI1241B9C8611', 0, 335.51628], ['SOLYIBD12A8C135045', 'Music is what we love', 'AR051KA1187B98B2FF', 0, 261.51138], ['SOWQTQZ12A58A7B63E', 'Streets On Fire (Explicit Album Version)', 'ARPFHN61187FB575F6', 0, 279.97995], ['SOUPIRU12A6D4FA1E1', 'Der Kleine Dompfaff', 'ARJIE2Y1187B994AB7', 0, 152.92036], ['SODREIN12A58A7F2E5', 'A Whiter Shade Of Pale (Live @ Fillmore West)', 'ARLTWXK1187FB5A3F8', 0

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [8]:
#song_data2 = ('vsd', 'sdf', 'sdf', 2008, 10.4)
cur.executemany(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [9]:
def get_artist_data(song_files):
    df5 = pd.DataFrame()
    for i in song_files:
        df4 = pd.read_json(i, lines = True)
        df5 = df5.append(df4)
    df5 = df5[['artist_id','artist_name','artist_location','artist_latitude','artist_longitude']]
    df5.values
    df5 = df5.drop_duplicates(keep = 'first')
    df5 = df5.values.tolist()
    return df5
artist_data = get_artist_data(song_files) 
print(artist_data)



[['AR8IEZO1187B99055E', 'Marc Shaiman', '', nan, nan], ['AR558FS1187FB45658', '40 Grit', '', nan, nan], ['ARVBRGZ1187FB4675A', 'Gwen Stefani', '', nan, nan], ['ARNF6401187FB57032', 'Sophie B. Hawkins', 'New York, NY [Manhattan]', 40.79086, -73.96644], ['AREVWGE1187B9B890A', 'Bitter End', 'Noci (BA)', -13.442, -41.9952], ['AR0IAWL1187B9A96D0', 'Danilo Perez', 'Panama', 8.4177, -80.11278], ['ARWB3G61187FB49404', 'Steve Morse', 'Hamilton, Ohio', nan, nan], ['ARULZCI1241B9C8611', 'Luna Orbit Project', '', nan, nan], ['AR051KA1187B98B2FF', 'Wilks', '', nan, nan], ['ARPFHN61187FB575F6', 'Lupe Fiasco', 'Chicago, IL', 41.88415, -87.63241], ['ARJIE2Y1187B994AB7', 'Line Renaud', '', nan, nan], ['ARLTWXK1187FB5A3F8', 'King Curtis', 'Fort Worth, TX', 32.74863, -97.32925], ['AR7SMBG1187B9B9066', 'Los Manolos', '', nan, nan], ['ARNNKDK1187B98BBD5', 'Jinx', 'Zagreb Croatia', 45.80726, 15.967600000000001], ['ARP6N5A1187B99D1A3', 'Mitch Ryder', 'Hamtramck, MI', nan, nan], ['ARBGXIG122988F409D', 'Steel 

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [10]:
cur.executemany(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [11]:
log_files = get_files('data/log_data')
print(log_files)

['/home/workspace/data/log_data/2018/11/2018-11-29-events.json', '/home/workspace/data/log_data/2018/11/2018-11-27-events.json', '/home/workspace/data/log_data/2018/11/2018-11-11-events.json', '/home/workspace/data/log_data/2018/11/2018-11-18-events.json', '/home/workspace/data/log_data/2018/11/2018-11-08-events.json', '/home/workspace/data/log_data/2018/11/2018-11-30-events.json', '/home/workspace/data/log_data/2018/11/2018-11-17-events.json', '/home/workspace/data/log_data/2018/11/2018-11-20-events.json', '/home/workspace/data/log_data/2018/11/2018-11-28-events.json', '/home/workspace/data/log_data/2018/11/2018-11-06-events.json', '/home/workspace/data/log_data/2018/11/2018-11-23-events.json', '/home/workspace/data/log_data/2018/11/2018-11-13-events.json', '/home/workspace/data/log_data/2018/11/2018-11-21-events.json', '/home/workspace/data/log_data/2018/11/2018-11-16-events.json', '/home/workspace/data/log_data/2018/11/2018-11-04-events.json', '/home/workspace/data/log_data/2018/11/

In [12]:
filepath = log_files[0]
print(filepath)

/home/workspace/data/log_data/2018/11/2018-11-29-events.json


In [13]:
df = pd.read_json(filepath, lines=True)
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Sydney Youngblood,Logged In,Jacob,M,53,Klein,238.07955,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,954,Ain't No Sunshine,200,1543449657796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
1,Gang Starr,Logged In,Layla,F,88,Griffin,151.92771,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,My Advice 2 You (Explicit),200,1543449690796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24
2,3OH!3,Logged In,Layla,F,89,Griffin,192.52200,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,My First Kiss (Feat. Ke$ha) [Album Version],200,1543449841796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24
3,RÃÂ¶yksopp,Logged In,Jacob,M,54,Klein,369.81506,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,954,The Girl and The Robot,200,1543449895796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
4,Kajagoogoo,Logged In,Layla,F,90,Griffin,223.55546,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,Too Shy,200,1543450033796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [14]:
df = df[df['page']=='NextSong']
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Sydney Youngblood,Logged In,Jacob,M,53,Klein,238.07955,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,954,Ain't No Sunshine,200,1543449657796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
1,Gang Starr,Logged In,Layla,F,88,Griffin,151.92771,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,My Advice 2 You (Explicit),200,1543449690796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24
2,3OH!3,Logged In,Layla,F,89,Griffin,192.52200,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,My First Kiss (Feat. Ke$ha) [Album Version],200,1543449841796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24
3,RÃÂ¶yksopp,Logged In,Jacob,M,54,Klein,369.81506,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,954,The Girl and The Robot,200,1543449895796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
4,Kajagoogoo,Logged In,Layla,F,90,Griffin,223.55546,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,Too Shy,200,1543450033796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24


In [15]:
df['ts'] = pd.to_datetime(df['ts'], unit='ms')
t = df
t.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Sydney Youngblood,Logged In,Jacob,M,53,Klein,238.07955,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,954,Ain't No Sunshine,200,2018-11-29 00:00:57.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
1,Gang Starr,Logged In,Layla,F,88,Griffin,151.92771,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,My Advice 2 You (Explicit),200,2018-11-29 00:01:30.796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24
2,3OH!3,Logged In,Layla,F,89,Griffin,192.52200,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,My First Kiss (Feat. Ke$ha) [Album Version],200,2018-11-29 00:04:01.796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24
3,RÃÂ¶yksopp,Logged In,Jacob,M,54,Klein,369.81506,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,954,The Girl and The Robot,200,2018-11-29 00:04:55.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
4,Kajagoogoo,Logged In,Layla,F,90,Griffin,223.55546,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,Too Shy,200,2018-11-29 00:07:13.796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24


In [16]:
s = t['ts']
result0 = s
result1 = s.dt.hour
result2 = s.dt.day
result3 = s.dt.week
result4 = s.dt.month
result5 = s.dt.year
result6 = s.dt.weekday
#print(result0)
time_data = [result0,result1, result2, result3, result4, result5, result6]
#print(time_data)
column_labels = ('timestamp', 'hour', 'day', 'week_of_year', 'month', 'year', 'weekday')
print(column_labels)

('timestamp', 'hour', 'day', 'week_of_year', 'month', 'year', 'weekday')


In [17]:
time_df = dict(zip(column_labels,time_data))
time_df =  pd.DataFrame.from_dict(time_df) 
time_df.head()

,timestamp,hour,day,week_of_year,month,year,weekday
0,2018-11-29 00:00:57.796,0,29,48,11,2018,3
1,2018-11-29 00:01:30.796,0,29,48,11,2018,3
2,2018-11-29 00:04:01.796,0,29,48,11,2018,3
3,2018-11-29 00:04:55.796,0,29,48,11,2018,3
4,2018-11-29 00:07:13.796,0,29,48,11,2018,3


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [18]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [19]:

user_df = df[['userId', 'firstName', 'lastName', 'gender', 'level']]
user_df

,userId,firstName,lastName,gender,level
0,73,Jacob,Klein,M,paid
1,24,Layla,Griffin,F,paid
2,24,Layla,Griffin,F,paid
3,73,Jacob,Klein,M,paid
4,24,Layla,Griffin,F,paid
5,24,Layla,Griffin,F,paid
6,73,Jacob,Klein,M,paid
7,73,Jacob,Klein,M,paid
8,24,Layla,Griffin,F,paid
9,24,Layla,Griffin,F,paid


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [20]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [21]:
#df.head()
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist
                              , row.length))
    results = cur.fetchone()
    #print(results)
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None
   # print(results)
    # insert songplay record
    songplay_data = (pd.to_datetime(row.ts, unit = "ms"), row.userId, row.level, songid, artistid, row.sessionId, row.location, row.userAgent)
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [22]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.